**Tema 4 - Metode de clasificare**

Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 6. Cel putin un set de date sa fie cu valori lipsa; pentru un alt set de date care are initial toate valorile, introduceti dvs. in mod artificial valori lipsa, suprascriind un anumit procent din valorile initiale (ex. `p=5%`, `p` parametru) cu `numpy.nan`. 

In [1]:
import numpy as np
import pandas as pd
import sklearn

print('At the moment the program was made:')
print('\tNumPy Version = 1.20.2')
print('\tScykit_learn Version = 0.24.1')
print('\tPandas Version = 1.2.3\n')

print(f'Current NumPy Version = {np.__version__}')
print(f'Current Sklearn Version = {sklearn.__version__}')
print(f'Current Pandas Version = {pd.__version__}')

At the moment the program was made:
	NumPy Version = 1.20.2
	Scykit_learn Version = 0.24.1
	Pandas Version = 1.2.3

Current NumPy Version = 1.20.2
Current Sklearn Version = 0.24.1
Current Pandas Version = 1.2.4


## Subpunctul 1
1. Aplicati o metoda de missing value imputation, unde este cazul; documentati metoda folosita.
*Resurse*: Pentru missing value imputation, puteti urmari [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html), [How to Handle Missing Data with Python](https://machinelearningmastery.com/handle-missing-data-python/), [fancyimpute](https://github.com/iskandr/fancyimpute), [missingpy](https://github.com/epsilon-machine/missingpy).

### Setul de date `Planning Relax`

In [18]:
pd_data: pd.DataFrame = pd.read_csv("data/planning_relax_data_set.txt", delimiter="\t", header=None)
pd_data = pd_data.drop(13, axis=1)
pd_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.17936,-0.20700,-0.209710,-0.097260,-0.119210,-0.17322,-0.280760,0.223170,0.418660,-0.032886,0.003383,-0.334250,1.0
1,-0.14659,-0.16494,0.249870,-0.073985,0.494940,-0.15633,-0.288910,0.500800,-0.045553,0.050759,-0.017511,0.066959,1.0
2,-0.13131,0.51816,0.432810,0.397890,0.043127,0.56320,-0.013246,-0.062833,-0.883390,-0.040303,0.088057,0.631200,2.0
3,0.64940,0.41878,0.148350,0.333490,-0.219740,0.39891,0.590650,0.077100,-1.107200,-0.367230,0.364680,0.571720,1.0
4,-0.40628,0.11503,0.073336,0.070066,-0.030920,0.15205,-0.191420,-0.580430,-0.102250,0.153750,-0.126950,0.184570,1.0


In [19]:
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       182 non-null    float64
 1   1       182 non-null    float64
 2   2       182 non-null    float64
 3   3       182 non-null    float64
 4   4       182 non-null    float64
 5   5       182 non-null    float64
 6   6       182 non-null    float64
 7   7       182 non-null    float64
 8   8       182 non-null    float64
 9   9       182 non-null    float64
 10  10      182 non-null    float64
 11  11      182 non-null    float64
 12  12      182 non-null    float64
dtypes: float64(13)
memory usage: 18.6 KB


In [20]:
pd_data: np.ndarray = pd_data.values
pd_data_original: np.ndarray = pd_data.copy()
assert not np.isnan(pd_data_original.sum()) # verificam ca nu e niciun nan 

Setul initial de date nu are valori lipsa, dar am implementat o functie care sa suprascrie un anumit procent din valorile initiale (dat ca parametru) folosind `numpy.nan`

In [21]:
def insert_nan(data: np.ndarray, percentage: int=15) -> np.ndarray:
    '''
    Function to insert Nan in a numpy array, given a percentage from the user.
    :param data: array that represents the data to be modified
    :param percentage: integer representing the percentage of numbers from the array to be set to Nan.
    :returns: the initial :param data: array with Nan values modified according to :param percentage:    
    '''
    assert(percentage >=0 and percentage <=100)
    numbers_to_be_modified: int = int(percentage / 100 * data.shape[0] * data.shape[1])
    random_lines: np.ndarray = np.random.choice(data.shape[0], numbers_to_be_modified)
    random_cols: np.ndarray = np.random.choice(data.shape[1], numbers_to_be_modified)
    data[random_lines, random_cols] = np.nan
    return data

pd_data: np.ndarray = insert_nan(pd_data)
print(f'DataSet after adding Nan: \n {pd_data}')
assert np.isnan(pd_data.sum()) # verificam ca e cel putin un nan

DataSet after adding Nan: 
 [[-0.17936   -0.207     -0.20971   ...  0.0033827 -0.33425    1.       ]
 [-0.14659   -0.16494    0.24987   ...        nan  0.066959   1.       ]
 [       nan  0.51816    0.43281   ...  0.088057   0.6312     2.       ]
 ...
 [ 0.29877   -0.17962    0.032859  ...  0.24683   -0.38916    2.       ]
 [-0.37812   -0.52287    0.21541   ...  0.007591   0.47603    1.       ]
 [ 0.37913   -0.046562  -0.12336   ...  0.17369    0.49063    2.       ]]


In [24]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 
imputer: IterativeImputer = IterativeImputer(max_iter=1000, random_state=0)
pd_data_sklearn: np.ndarray = imputer.fit_transform(pd_data)
print(f'DataSet after missing values imputation: \n {pd_data_sklearn}')

DataSet after missing values imputation: 
 [[-0.17936    -0.207      -0.20971    ...  0.0033827  -0.33425
   1.        ]
 [-0.14659    -0.16494     0.24987    ... -0.01782124  0.066959
   1.        ]
 [-0.1293628   0.51816     0.43281    ...  0.088057    0.6312
   2.        ]
 ...
 [ 0.29877    -0.17962     0.032859   ...  0.24683    -0.38916
   2.        ]
 [-0.37812    -0.52287     0.21541    ...  0.007591    0.47603
   1.        ]
 [ 0.37913    -0.046562   -0.12336    ...  0.17369     0.49063
   2.        ]]


In [25]:
np.linalg.norm(pd_data_sklearn - pd_data_original)

3.6383655001150568

### Setul de date `Conventional Social Media Movies`

In [26]:
data: pd.DataFrame = pd.read_excel('data/2014 and 2015 CSM dataset.xlsx', index_col=None, header=None)
data = data.drop(0, axis=0)
data = data.drop(0, axis=1)

In [27]:
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
1,2014,6.3,8,9130,4000000,45,1,0,3280543,4632,425,636,1120000
2,2014,7.1,1,192000000,50000000,3306,2,2,583289,3465,61,186,12350000
3,2014,6.2,1,30700000,28000000,2872,1,0,304861,328,34,47,483000
4,2014,6.3,1,106000000,110000000,3470,2,0,452917,2429,132,590,568000
5,2014,4.7,8,17300000,3500000,2310,2,0,3145573,12163,610,1082,1923800


In [28]:
data: np.ndarray = data.values
print(data)
data_original: np.ndarray = data.copy()
assert np.isnan(data.sum()) # verificam ca e cel putin un nan

[[2014 6.3 8 ... 425 636 1120000]
 [2014 7.1 1 ... 61 186 12350000]
 [2014 6.2 1 ... 34 47 483000]
 ...
 [2015 5.4 8 ... 325 409 nan]
 [2015 5.4 1 ... 67 201 nan]
 [2015 4.4 15 ... 431 606 nan]]


In [29]:
!pip install missingpy

In [30]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [31]:
from missingpy import MissForest

imputer: MissForest = MissForest()
data: np.ndarray = imputer.fit_transform(data)

Iteration: 0
Iteration: 1
Iteration: 2


In [32]:
print(f'Data original:\n {data_original}')
print(f'\nData filled:\n {data}')

assert not np.isnan(data.sum()) # verificam ca nu exista niciun nan

Data original:
 [[2014 6.3 8 ... 425 636 1120000]
 [2014 7.1 1 ... 61 186 12350000]
 [2014 6.2 1 ... 34 47 483000]
 ...
 [2015 5.4 8 ... 325 409 nan]
 [2015 5.4 1 ... 67 201 nan]
 [2015 4.4 15 ... 431 606 nan]]

Data filled:
 [[2.01400000e+03 6.30000000e+00 8.00000000e+00 ... 4.25000000e+02
  6.36000000e+02 1.12000000e+06]
 [2.01400000e+03 7.10000000e+00 1.00000000e+00 ... 6.10000000e+01
  1.86000000e+02 1.23500000e+07]
 [2.01400000e+03 6.20000000e+00 1.00000000e+00 ... 3.40000000e+01
  4.70000000e+01 4.83000000e+05]
 ...
 [2.01500000e+03 5.40000000e+00 8.00000000e+00 ... 3.25000000e+02
  4.09000000e+02 1.76244981e+06]
 [2.01500000e+03 5.40000000e+00 1.00000000e+00 ... 6.70000000e+01
  2.01000000e+02 6.42576860e+05]
 [2.01500000e+03 4.40000000e+00 1.50000000e+01 ... 4.31000000e+02
  6.06000000e+02 2.22139017e+06]]


## Subpunctul 2
2. Pentru fiecare set de date aplicati 5 modele de clasificare din scikit learn. Pentru fiecare raportati: acuratete, precision, recall, scorul F1 - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics), [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) - folosind 5 fold cross validation. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare. Rularile se vor face cu valori fixate ale hiperparametrilor. 

### Setul de date `Planning Relax`

In [33]:
pr_data: pd.DataFrame = pd.read_csv("data/planning_relax_data_set.txt", delimiter="\t", header=None)
pr_data = pr_data.drop(13, axis=1)
pr_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.17936,-0.20700,-0.209710,-0.097260,-0.119210,-0.17322,-0.280760,0.223170,0.418660,-0.032886,0.003383,-0.334250,1.0
1,-0.14659,-0.16494,0.249870,-0.073985,0.494940,-0.15633,-0.288910,0.500800,-0.045553,0.050759,-0.017511,0.066959,1.0
2,-0.13131,0.51816,0.432810,0.397890,0.043127,0.56320,-0.013246,-0.062833,-0.883390,-0.040303,0.088057,0.631200,2.0
3,0.64940,0.41878,0.148350,0.333490,-0.219740,0.39891,0.590650,0.077100,-1.107200,-0.367230,0.364680,0.571720,1.0
4,-0.40628,0.11503,0.073336,0.070066,-0.030920,0.15205,-0.191420,-0.580430,-0.102250,0.153750,-0.126950,0.184570,1.0


In [34]:
pr_X: np.ndarray = pr_data.iloc[:, :-1].values
pr_y: np.ndarray = pr_data.iloc[:, -1].values

In [35]:
print(pr_X)
print(pr_y)

[[-0.17936   -0.207     -0.20971   ... -0.032886   0.0033827 -0.33425  ]
 [-0.14659   -0.16494    0.24987   ...  0.050759  -0.017511   0.066959 ]
 [-0.13131    0.51816    0.43281   ... -0.040303   0.088057   0.6312   ]
 ...
 [ 0.29877   -0.17962    0.032859  ... -0.41113    0.24683   -0.38916  ]
 [-0.37812   -0.52287    0.21541   ... -0.039117   0.007591   0.47603  ]
 [ 0.37913   -0.046562  -0.12336   ... -0.24229    0.17369    0.49063  ]]
[1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2.
 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2.]


In [116]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from typing import Union, List, Dict, Any

ModelEstimator = Union[KNeighborsClassifier, DecisionTreeClassifier, SVC, SGDClassifier, GaussianProcessClassifier, RandomForestClassifier]
Kernels = Union[RBF]

In [48]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn import metrics

  
def calculate_scores_micro(model:ModelEstimator, X: np.ndarray, y: np.ndarray) -> None:
    '''
    Function to print the mean scores for train data set and test data set using the estimator given as parameter.
    :param model: Classifier from sklearn
    :param X: array representing the dimensions and features of the data set
    :param y: array representing the classes of the data set
    :returns: None
    '''
    scores: Dict[str, List[float]] = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'precision', 'recall', 'f1'], return_train_score=True)
    train_accuracy: float = scores['train_accuracy'].mean()
    test_accuracy: float = scores['test_accuracy'].mean()
    train_precision: float = scores['train_precision'].mean()
    test_precision: float = scores['test_precision'].mean()
    train_recall: float = scores['train_recall'].mean()
    test_recall: float = scores['test_recall'].mean()
    train_f1_score: float = scores['train_f1'].mean()
    test_f1_score: float = scores['test_f1'].mean()
    print(f'Train accuracy: {train_accuracy}')
    print(f'Test accuracy: {test_accuracy}')
    print(f'Train precision: {train_precision}')
    print(f'Test precision: {test_precision}')
    print(f'Train recall: {train_recall}')
    print(f'Test recall: {test_recall}')
    print(f'Train F1: {train_f1_score}')
    print(f'Test F1: {test_f1_score}')

* Folosind `KNeighborsClassifier`

In [49]:
model: ModelEstimator = KNeighborsClassifier(n_neighbors=3)
calculate_scores_micro(model=model, X=pr_X, y=pr_y)

Train accuracy: 0.8186679263108172
Test accuracy: 0.6376876876876877
Train precision: 0.8283701291123394
Test precision: 0.7074860354699064
Train recall: 0.9423076923076923
Test recall: 0.8384615384615384
Train F1: 0.8814538535510122
Test F1: 0.7668308784169008


* Folosind `DecisionTreeClassifier`

In [50]:
classifier: ModelEstimator = DecisionTreeClassifier()
calculate_scores_micro(model=classifier, X=pr_X, y=pr_y)

Train accuracy: 1.0
Test accuracy: 0.5722222222222222
Train precision: 1.0
Test precision: 0.6919131359524068
Train recall: 1.0
Test recall: 0.7153846153846153
Train F1: 1.0
Test F1: 0.7017023633564987


* Folosind `SupportVectorMachines`

In [51]:
classifier: ModelEstimator = SVC(kernel='linear')
calculate_scores_micro(model=classifier, X=pr_X, y=pr_y)

Train accuracy: 0.7142938119981105
Test accuracy: 0.7144144144144144
Train precision: 0.7142938119981105
Test precision: 0.7144144144144144
Train recall: 1.0
Test recall: 1.0
Train F1: 0.833336546184739
Test F1: 0.8333845366103431


* Folosind `SochasticGradientDescent`

In [52]:
classifier: ModelEstimator = SGDClassifier(loss='hinge', penalty='l2')
calculate_scores_micro(model=classifier, X=pr_X, y=pr_y)

Train accuracy: 0.6512706660368446
Test accuracy: 0.6144144144144145
Train precision: 0.7265583746926186
Test precision: 0.6997511578074238
Train recall: 0.826923076923077
Test recall: 0.8
Train F1: 0.7655602898812913
Test F1: 0.7309022833939778


* Folosind `RBF`

In [53]:
kernel: Kernels = 1.0 * RBF(1.0)
gpc: ModelEstimator = GaussianProcessClassifier(kernel=kernel)
calculate_scores_micro(model=gpc, X=pr_X, y=pr_y)

Train accuracy: 0.7142938119981105
Test accuracy: 0.7144144144144144
Train precision: 0.7142938119981105
Test precision: 0.7144144144144144
Train recall: 1.0
Test recall: 1.0
Train F1: 0.833336546184739
Test F1: 0.8333845366103431


### Setul de date `Divorce`

In [54]:
divorce_data: pd.DataFrame = pd.read_excel('data/divorce.xlsx', index_col=None, header=None)
divorce_data = divorce_data.drop(0, axis=0)

In [55]:
# divorce_data.head()
print(divorce_data)

    0  1  2  3  4  5  6  7  8  9   ... 45 46 47 48 49 50 51 52 53 54
1    2  2  4  1  0  0  0  0  0  0  ...  2  1  3  3  3  2  3  2  1  1
2    4  4  4  4  4  0  0  4  4  4  ...  2  2  3  4  4  4  4  2  2  1
3    2  2  2  2  1  3  2  1  1  2  ...  3  2  3  1  1  1  2  2  2  1
4    3  2  3  2  3  3  3  3  3  3  ...  2  2  3  3  3  3  2  2  2  1
5    2  2  1  1  1  1  0  0  0  0  ...  2  1  2  3  2  2  2  1  0  1
..  .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
166  0  0  0  0  0  0  0  0  0  0  ...  1  0  4  1  1  4  2  2  2  0
167  0  0  0  0  0  0  0  0  0  0  ...  4  1  2  2  2  2  3  2  2  0
168  1  1  0  0  0  0  0  0  0  1  ...  3  0  2  0  1  1  3  0  0  0
169  0  0  0  0  0  0  0  0  0  0  ...  3  3  2  2  3  2  4  3  1  0
170  0  0  0  0  0  0  0  1  0  0  ...  3  4  4  0  1  3  3  3  1  0

[170 rows x 55 columns]


In [56]:
divorce_X: np.ndarray = divorce_data.iloc[:, :-1].values
divorce_y: np.ndarray = divorce_data.iloc[:, -1].values
divorce_y = divorce_y.astype('int')

* Folosind `KNeighborsClassifier`

In [57]:
model: ModelEstimator = KNeighborsClassifier(n_neighbors=3)
calculate_scores_micro(model=model, X=divorce_X, y=divorce_y)

Train accuracy: 0.9779411764705882
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 1.0
Train recall: 0.9553555750658473
Test recall: 0.9529411764705882
Train F1: 0.9770271833429728
Test F1: 0.9733333333333334


* Folosind `DecisionTreeClassifier`

In [58]:
classifier: ModelEstimator = DecisionTreeClassifier()
calculate_scores_micro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9529411764705882
Train precision: 1.0
Test precision: 0.9666666666666668
Train recall: 1.0
Test recall: 0.9411764705882353
Train F1: 1.0
Test F1: 0.9480382497826717


* Folosind `SupportVectorMachines`

In [59]:
classifier: ModelEstimator = SVC(kernel='linear')
calculate_scores_micro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 1.0
Train recall: 1.0
Test recall: 0.9529411764705882
Train F1: 1.0
Test F1: 0.9733333333333334


* Folosind `SochasticGradientDescent`

In [60]:
classifier: ModelEstimator = SGDClassifier(loss='hinge', penalty='l2')
calculate_scores_micro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9588235294117646
Train precision: 1.0
Test precision: 0.966013071895425
Train recall: 1.0
Test recall: 0.9529411764705882
Train F1: 1.0
Test F1: 0.9558441558441558


* Folosind `RBF`

In [61]:
kernel: Kernels = 1.0 * RBF(1.0)
gpc: ModelEstimator = GaussianProcessClassifier(kernel=kernel)
calculate_scores_micro(model=gpc, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 1.0
Train recall: 1.0
Test recall: 0.9529411764705882
Train F1: 1.0
Test F1: 0.9733333333333334


### Setul de date `HCV`

In [70]:
data_HCV: pd.DataFrame = pd.read_csv('data/hcv_data.csv')
data_HCV = data_HCV.drop('Unnamed: 0', axis=1)
data_HCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    object 
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    object 
 3   ALB       614 non-null    float64
 4   ALP       597 non-null    float64
 5   ALT       614 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      605 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      614 non-null    float64
dtypes: float64(10), int64(1), object(2)
memory usage: 62.6+ KB


In [71]:
from sklearn.preprocessing import LabelEncoder

labelencoder: LabelEncoder = LabelEncoder()

data_HCV.Category: pd.Series = data_HCV.Category.astype('category')
data_HCV.Category = data_HCV.Category.cat.codes
data_HCV.Sex: pd.Series = data_HCV.Sex.astype('category')
data_HCV.Sex = data_HCV.Sex.cat.codes
data_HCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    int8   
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    int8   
 3   ALB       614 non-null    float64
 4   ALP       597 non-null    float64
 5   ALT       614 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      605 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      614 non-null    float64
dtypes: float64(10), int64(1), int8(2)
memory usage: 54.2 KB


In [72]:
data_HCV: pd.DataFrame = data_HCV.fillna(data_HCV.mean())
data_HCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    int8   
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    int8   
 3   ALB       615 non-null    float64
 4   ALP       615 non-null    float64
 5   ALT       615 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      615 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      615 non-null    float64
dtypes: float64(10), int64(1), int8(2)
memory usage: 54.2 KB


In [73]:
HCV_X: np.ndarray = data_HCV.iloc[:, 1:].values
HCV_y: np.ndarray = data_HCV.iloc[:, 0].values

In [74]:
np.any(np.isnan(data_HCV))

False

In [75]:
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [76]:
def calculate_scores_macro(model: ModelEstimator, X:np.ndarray, y:np.ndarray) -> None:
    '''
    Function to print the mean scores for train data set and test data set using the estimator given as parameter.
    :param model: Classifier from sklearn
    :param X: array representing the dimensions and features of the data set
    :param y: array representing the classes of the data set
    :returns: None
    '''
    scores: Dict[str, List[float]] = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=True)
    train_accuracy: float = scores['train_accuracy'].mean()
    test_accuracy: float = scores['test_accuracy'].mean()
    train_precision: float = scores['train_precision_macro'].mean()
    test_precision: float = scores['test_precision_macro'].mean()
    train_recall: float = scores['train_recall_macro'].mean()
    test_recall: float = scores['test_recall_macro'].mean()
    train_f1_score: float = scores['train_f1_macro'].mean()
    test_f1_score: float = scores['test_f1_macro'].mean()
    print(f'Train accuracy: {train_accuracy}')
    print(f'Test accuracy: {test_accuracy}')
    print(f'Train precision: {train_precision}')
    print(f'Test precision: {test_precision}')
    print(f'Train recall: {train_recall}')
    print(f'Test recall: {test_recall}')
    print(f'Train F1: {train_f1_score}')
    print(f'Test F1: {test_f1_score}')

* Folosind `KNeighborsClassifier`

In [77]:
model: ModelEstimator = KNeighborsClassifier(n_neighbors=3)
calculate_scores_macro(model=model, X=HCV_X, y=HCV_y)

Train accuracy: 0.9443089430894309
Test accuracy: 0.9008130081300812
Train precision: 0.8095025426719882
Test precision: 0.524628733997155
Train recall: 0.6235858862253356
Test recall: 0.39649409275260095
Train F1: 0.6855780932171947
Test F1: 0.4394494925234267


* Folosind `DecisionTreeClassifier`

In [78]:
classifier: ModelEstimator = DecisionTreeClassifier()
calculate_scores_macro(model=classifier, X=HCV_X, y=HCV_y)

Train accuracy: 1.0
Test accuracy: 0.9056910569105691
Train precision: 1.0
Test precision: 0.5318983349926747
Train recall: 1.0
Test recall: 0.5504594133897609
Train F1: 1.0
Test F1: 0.5326240624929708


* Folosind `SupportVectorMachines`

In [79]:
classifier: ModelEstimator = SVC(kernel='linear')
calculate_scores_macro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 0.980952380952381
Train recall: 1.0
Test recall: 0.9764705882352942
Train F1: 1.0
Test F1: 0.9761403508771931


* Folosind `SochasticGradientDescent`

In [80]:
classifier: ModelEstimator = SGDClassifier(loss='hinge', penalty='l2')
calculate_scores_macro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9705882352941175
Train precision: 1.0
Test precision: 0.9750700280112043
Train recall: 1.0
Test recall: 0.9709150326797384
Train F1: 1.0
Test F1: 0.9702529049897471


* Folosind `RBF`

In [81]:
kernel: Kernels = 1.0 * RBF(1.0)
gpc: ModelEstimator = GaussianProcessClassifier(kernel=kernel)
calculate_scores_macro(model=gpc, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 0.980952380952381
Train recall: 1.0
Test recall: 0.9764705882352942
Train F1: 1.0
Test F1: 0.9761403508771931


### Setul de date `Conventional Social Media Movies`

In [85]:
CSM_data: pd.DataFrame = pd.read_excel('data/2014 and 2015 CSM dataset.xlsx', index_col=None, header=None)
CSM_data = CSM_data.drop(0, axis=0)
CSM_data = CSM_data.drop(0, axis=1)

In [86]:
CSM_data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
1,2014,6.3,8,9130,4000000,45,1,0,3280543,4632,425,636,1120000
2,2014,7.1,1,192000000,50000000,3306,2,2,583289,3465,61,186,12350000
3,2014,6.2,1,30700000,28000000,2872,1,0,304861,328,34,47,483000
4,2014,6.3,1,106000000,110000000,3470,2,0,452917,2429,132,590,568000
5,2014,4.7,8,17300000,3500000,2310,2,0,3145573,12163,610,1082,1923800


In [87]:
from missingpy import MissForest

imputer: MissForest = MissForest()
CSM_data: np.ndarray = imputer.fit_transform(CSM_data)
assert not np.isnan(CSM_data.sum()) # verificam ca nu exista niciun nan

Iteration: 0
Iteration: 1
Iteration: 2


In [88]:
CSM_y: np.ndarray = CSM_data[:, 2]
CSM_X: np.ndarray = CSM_data[:, [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

Datorita diferentelor foarte mari de valori din tabel, am decis sa facem o scalare a valorilor, pentru o clasificare mai buna.

In [90]:
from sklearn.preprocessing import MinMaxScaler
scaler: MinMaxScaler = MinMaxScaler()
scaler.fit(CSM_X)
CSM_X: np.ndarray = scaler.transform(CSM_X)
print(CSM_X)

[[0.00000000e+00 5.71428571e-01 1.03577381e-05 ... 3.04441261e-02
  1.65784741e-02 3.60609875e-02]
 [0.00000000e+00 7.14285714e-01 2.98597617e-01 ... 4.36962751e-03
  4.84842166e-03 3.97981252e-01]
 [0.00000000e+00 5.53571429e-01 4.77412876e-02 ... 2.43553009e-03
  1.22513881e-03 1.55317614e-02]
 ...
 [1.00000000e+00 4.10714286e-01 1.58593611e-02 ... 2.32808023e-02
  1.06613143e-02 5.55760179e-02]
 [1.00000000e+00 4.10714286e-01 1.91253145e-02 ... 4.79942693e-03
  5.23942340e-03 2.75920890e-02]
 [1.00000000e+00 2.32142857e-01 3.51440386e-02 ... 3.08739255e-02
  1.57964706e-02 5.95488666e-02]]


In [91]:
from sklearn.model_selection import ShuffleSplit
def our_function_macro(model: ModelEstimator, X: np.ndarray, y:np.ndarray) -> None:
    '''
    Function to print the mean scores for train data set and test data set using the estimator given as parameter.
    :param model: Classifier from sklearn
    :param X: array representing the dimensions and features of the data set
    :param y: array representing the classes of the data set
    :returns: None
    '''
    cv: ShuffleSplit = ShuffleSplit(n_splits=10, test_size=1/3, random_state=0)
    scores: Dict[str, List[float]] = cross_validate(model, X, y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=True)
    train_accuracy: float = scores['train_accuracy'].mean()
    test_accuracy: float = scores['test_accuracy'].mean()
    train_precision: float = scores['train_precision_macro'].mean()
    test_precision: float = scores['test_precision_macro'].mean()
    train_recall: float = scores['train_recall_macro'].mean()
    test_recall: float = scores['test_recall_macro'].mean()
    train_f1_score: float = scores['train_f1_macro'].mean()
    test_f1_score: float = scores['test_f1_macro'].mean()
    print(f'Train accuracy: {train_accuracy}')
    print(f'Test accuracy: {test_accuracy}')
    print(f'Train precision: {train_precision}')
    print(f'Test precision: {test_precision}')
    print(f'Train recall: {train_recall}')
    print(f'Test recall: {test_recall}')
    print(f'Train F1: {train_f1_score}')
    print(f'Test F1: {test_f1_score}')

* Folosind `KNeighborsClassifier`

In [92]:
model: ModelEstimator = KNeighborsClassifier(n_neighbors=5)
# our_function_macro(model=model, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=model, X=CSM_X, y=CSM_y)

Train accuracy: 0.5043301997649824
Test accuracy: 0.24625346901017578
Train precision: 0.392661646827815
Test precision: 0.07360697640025192
Train recall: 0.2540496501099038
Test recall: 0.10624363291029956
Train F1: 0.2545331090117121
Test F1: 0.08236584518443002


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `DecisionTreeClassifier`

In [93]:
classifier: ModelEstimator = DecisionTreeClassifier()
# our_function_macro(model=classifier, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=classifier, X=CSM_X, y=CSM_y)

Train accuracy: 1.0
Test accuracy: 0.2074930619796485
Train precision: 1.0
Test precision: 0.09949101403646858
Train recall: 1.0
Test recall: 0.1114601335813457
Train F1: 1.0
Test F1: 0.10171075524995406


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `SupportVectorMachines`

In [94]:
classifier: ModelEstimator = SVC()
# our_function_macro(model=classifier, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=classifier, X=CSM_X, y=CSM_y)

Train accuracy: 0.42748531139835483
Test accuracy: 0.346068455134135
Train precision: 0.1198779394870189
Test precision: 0.11725339919847928
Train recall: 0.16517263904156082
Test recall: 0.15703358370025036
Train F1: 0.13495435628490676
Test F1: 0.11100186587259989


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `SochasticGradientDescent`

In [95]:
classifier: ModelEstimator = SGDClassifier(loss='hinge', penalty='l2')
# our_function_macro(model=classifier, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=classifier, X=CSM_X, y=CSM_y)

Train accuracy: 0.37011750881316097
Test accuracy: 0.3288621646623497
Train precision: 0.26842517969878876
Test precision: 0.1628096584377286
Train recall: 0.2797610519492126
Test recall: 0.21196512129845463
Train F1: 0.20791695475809227
Test F1: 0.1645262091322784


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `RBF`

In [96]:
kernel: Kernels = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e10))
gpc: ModelEstimator = GaussianProcessClassifier(kernel=kernel)
# our_function_macro(model=gpc, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=gpc, X=CSM_X, y=CSM_y)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Train accuracy: 0.42744418331374856
Test accuracy: 0.3505087881591119
Train precision: 0.1886414343950713
Test precision: 0.13116711776928497
Train recall: 0.19593750908761481
Test recall: 0.1773443840110507
Train F1: 0.1786979932184969
Test F1: 0.13639727695641674


## Subpunctul 3
3. Raportati performanta fiecarui model, folosind 5 fold cross validation. Pentru fiecare din cele 5 rulari, cautati hiperparametrii optimi folosind 4-fold cross validation. Performanta modelului va fi raportata ca medie a celor  5 rulari. 
    
    *Observatie:* la fiecare din cele 5 rulari, hiperparametrii optimi pot diferi, din cauza datelor utilizate pentru antrenare/validare. 

### Setul de date `Planning Relax`

In [101]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [102]:
pr_data: pd.DataFrame = pd.read_csv("data/planning_relax_data_set.txt", delimiter="\t", header=None)
pr_data = pr_data.drop(13, axis=1)
pr_X: np.ndarray = pr_data.iloc[:, :-1].values
pr_y: np.ndarray = pr_data.iloc[:, -1].values

In [111]:
def calculate_performance(model: ModelEstimator, parameter_grid: Dict[str, List[Any]], X: np.ndarray, y: np.ndarray) -> pd.DataFrame:
    '''
    Function to calculate the performance of an estimator, using GridSearchCV for finding the best hyperparameters for the estimator.
    :param model: Classifier from sklearn
    :param parameter_grid: Dictionary with keys being the parameters of the estimator and values being the values to be tested
    :param X: array representing the dimensions and features of the data set
    :param y: array representing the classes of the data set
    :returns: pd.DataFrame containing the cross validation results of the grid search
    '''
    grid_search: GridSearchCV = GridSearchCV(estimator = model, param_grid = parameter_grid, scoring = 'accuracy', cv=5, return_train_score=True)
    scores: Dict[str, List[float]] = cross_val_score(grid_search, X, y, cv=5)
    print(f'Accuracy: {scores.mean()}')
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/5)
    grid_search.fit(X_train, y_train)
    df_grid_search: pd.DataFrame = pd.DataFrame(grid_search.cv_results_)
    return df_grid_search

* Folosind `KNeighborsClassifier`

In [112]:
parameter_grid: Dict[str, List[Any]] = {'n_neighbors': list(range(1, 10)), 'p':[1, 2, 3, 10]}
df_grid_search: pd.DataFrame = calculate_performance(model=KNeighborsClassifier(), parameter_grid=parameter_grid, X=pr_X, y=pr_y)
df_grid_search.head()

Accuracy: 0.6761261261261262


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000592,0.000484,0.001396,0.000489,1,1,"{'n_neighbors': 1, 'p': 1}",0.586207,0.551724,0.448276,...,0.551724,0.068966,36,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.000598,0.000488,0.001197,0.000746,1,2,"{'n_neighbors': 1, 'p': 2}",0.586207,0.620690,0.517241,...,0.586207,0.065426,35,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,0.000399,0.000489,0.002799,0.000402,1,3,"{'n_neighbors': 1, 'p': 3}",0.586207,0.586207,0.586207,...,0.613793,0.055172,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.000201,0.000403,0.002714,0.000397,1,10,"{'n_neighbors': 1, 'p': 10}",0.586207,0.655172,0.655172,...,0.662069,0.045747,28,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.000399,0.000489,0.001396,0.000489,2,1,"{'n_neighbors': 2, 'p': 1}",0.689655,0.758621,0.724138,...,0.703448,0.046775,17,0.784483,0.793103,0.775862,0.793103,0.767241,0.782759,0.010053


* Folosind `DecisionTreeClassifier`

In [113]:
parameter_grid: Dict[str, List[Any]] = {'criterion':['gini','entropy'], 'max_depth': list(range(4, 10))}
df_grid_search: pd.DataFrame = calculate_performance(model=DecisionTreeClassifier(), parameter_grid=parameter_grid, X=pr_X, y=pr_y)
df_grid_search.head()

Accuracy: 0.6986486486486486


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000798,0.000399,0.000199,0.000399,gini,4,"{'criterion': 'gini', 'max_depth': 4}",0.689655,0.689655,0.620690,...,0.675862,0.035166,1,0.879310,0.818966,0.836207,0.801724,0.793103,0.825862,0.030552
1,0.000798,0.000399,0.000199,0.000399,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.655172,0.689655,0.586207,...,0.662069,0.040213,6,0.931034,0.836207,0.879310,0.853448,0.810345,0.862069,0.041163
2,0.000598,0.000488,0.000399,0.000488,gini,6,"{'criterion': 'gini', 'max_depth': 6}",0.655172,0.724138,0.586207,...,0.668966,0.051609,3,0.956897,0.844828,0.939655,0.896552,0.827586,0.893103,0.050738
3,0.000598,0.000489,0.000399,0.000489,gini,7,"{'criterion': 'gini', 'max_depth': 7}",0.655172,0.689655,0.655172,...,0.675862,0.027586,1,0.974138,0.844828,0.956897,0.965517,0.836207,0.915517,0.061540
4,0.000798,0.000399,0.000199,0.000399,gini,8,"{'criterion': 'gini', 'max_depth': 8}",0.620690,0.689655,0.620690,...,0.668966,0.041379,3,1.000000,0.896552,0.974138,0.991379,0.879310,0.948276,0.050267


* Folosind `SupportVectorMachines`

In [114]:
parameter_grid: Dict[str, List[Any]] = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001], 'kernel':['rbf']}
df_grid_search: pd.DataFrame = calculate_performance(model=SVC(), parameter_grid=parameter_grid, X=pr_X, y=pr_y)
df_grid_search.head()

Accuracy: 0.7090090090090091


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.001003,0.000013,0.000593,0.000484,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.724138,0.724138,...,0.724138,0.0,1,0.724138,0.724138,0.724138,0.724138,0.724138,0.724138,0.0
1,0.000988,0.000621,0.000200,0.000399,0.1,0.1,rbf,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",0.724138,0.724138,...,0.724138,0.0,1,0.724138,0.724138,0.724138,0.724138,0.724138,0.724138,0.0
2,0.000792,0.000396,0.000990,0.000014,0.1,0.01,rbf,"{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}",0.724138,0.724138,...,0.724138,0.0,1,0.724138,0.724138,0.724138,0.724138,0.724138,0.724138,0.0
3,0.000991,0.000013,0.000398,0.000488,0.1,0.001,rbf,"{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}",0.724138,0.724138,...,0.724138,0.0,1,0.724138,0.724138,0.724138,0.724138,0.724138,0.724138,0.0
4,0.000389,0.000476,0.000597,0.000488,0.1,0.0001,rbf,"{'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}",0.724138,0.724138,...,0.724138,0.0,1,0.724138,0.724138,0.724138,0.724138,0.724138,0.724138,0.0


* Folosind `SochasticGradientDescent`

In [115]:
parameter_grid: Dict[str, List[Any]] = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'penalty':['l2', 'l1', 'elasticnet']}
df_grid_search: pd.DataFrame = calculate_performance(model=SGDClassifier(), parameter_grid=parameter_grid, X=pr_X, y=pr_y)
df_grid_search.head()

Accuracy: 0.7144144144144144


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000799,3.992805e-04,0.000200,0.000399,0.0001,l2,"{'alpha': 0.0001, 'penalty': 'l2'}",0.758621,0.689655,0.551724,...,0.696552,0.076797,14,0.732759,0.637931,0.620690,0.732759,0.724138,0.689655,0.049672
1,0.000996,1.154301e-06,0.000000,0.000000,0.0001,l1,"{'alpha': 0.0001, 'penalty': 'l1'}",0.620690,0.620690,0.689655,...,0.551724,0.123369,17,0.663793,0.706897,0.612069,0.586207,0.568966,0.627586,0.050972
2,0.000798,3.988287e-04,0.000200,0.000399,0.0001,elasticnet,"{'alpha': 0.0001, 'penalty': 'elasticnet'}",0.482759,0.586207,0.586207,...,0.613793,0.090972,16,0.646552,0.500000,0.637931,0.698276,0.689655,0.634483,0.071213
3,0.000798,3.988981e-04,0.000199,0.000399,0.001,l2,"{'alpha': 0.001, 'penalty': 'l2'}",0.724138,0.724138,0.724138,...,0.731034,0.013793,2,0.732759,0.741379,0.732759,0.732759,0.724138,0.732759,0.005452
4,0.000997,1.907349e-07,0.000000,0.000000,0.001,l1,"{'alpha': 0.001, 'penalty': 'l1'}",0.758621,0.689655,0.724138,...,0.731034,0.025805,2,0.741379,0.724138,0.750000,0.732759,0.741379,0.737931,0.008791


* Folosind `RandomForrestClassifier`

In [117]:
parameter_grid: Dict[str, List[Any]] = {'n_estimators':[10, 20, 30, 100], 'criterion':['gini','entropy']}
df_grid_search: pd.DataFrame = calculate_performance(model=RandomForestClassifier(), parameter_grid=parameter_grid, X=pr_X, y=pr_y)
df_grid_search.head()

Accuracy: 0.7034534534534534


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.010214,0.000370,0.001589,0.000470,gini,10,"{'criterion': 'gini', 'n_estimators': 10}",0.689655,0.586207,0.655172,...,0.662069,0.045747,7,0.965517,0.982759,0.974138,0.965517,0.965517,0.970690,0.006897
1,0.020541,0.000480,0.002000,0.000026,gini,20,"{'criterion': 'gini', 'n_estimators': 20}",0.689655,0.655172,0.689655,...,0.675862,0.016893,4,1.000000,0.974138,1.000000,0.982759,0.982759,0.987931,0.010345
2,0.030737,0.001172,0.002799,0.000402,gini,30,"{'criterion': 'gini', 'n_estimators': 30}",0.655172,0.620690,0.689655,...,0.668966,0.027586,5,0.982759,1.000000,1.000000,1.000000,0.991379,0.994828,0.006897
3,0.099329,0.001727,0.007569,0.000488,gini,100,"{'criterion': 'gini', 'n_estimators': 100}",0.689655,0.689655,0.655172,...,0.682759,0.013793,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.011832,0.001138,0.001312,0.000567,entropy,10,"{'criterion': 'entropy', 'n_estimators': 10}",0.655172,0.724138,0.689655,...,0.696552,0.025805,2,0.965517,0.974138,0.974138,0.982759,0.982759,0.975862,0.006451


### Setul de date `Divorce`

In [118]:
divorce_data: pd.DataFrame = pd.read_excel('data/divorce.xlsx', index_col=None, header=None)
divorce_data = divorce_data.drop(0, axis=0)
divorce_X: np.ndarray = divorce_data.iloc[:, :-1].values
divorce_y: np.ndarray = divorce_data.iloc[:, -1].values
divorce_y = divorce_y.astype('int')

* Folosind `KNeighborsClassifier`

In [120]:
parameter_grid: Dict[str, List[Any]] = {'n_neighbors': list(range(1, 10)), 'p':[1, 2, 3, 10]}
df_grid_search: pd.DataFrame = calculate_performance(model=KNeighborsClassifier(), parameter_grid=parameter_grid, X=divorce_X, y=divorce_y)
df_grid_search.head()

Accuracy: 0.9764705882352942


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000199,0.000398,0.001190,0.000402,1,1,"{'n_neighbors': 1, 'p': 1}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.000812,0.000406,0.000997,0.000002,1,2,"{'n_neighbors': 1, 'p': 2}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,0.000199,0.000398,0.004793,0.000402,1,3,"{'n_neighbors': 1, 'p': 3}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.000399,0.000489,0.004385,0.000484,1,10,"{'n_neighbors': 1, 'p': 10}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.000399,0.000489,0.001196,0.000397,2,1,"{'n_neighbors': 2, 'p': 1}",0.964286,1.0,1.0,...,0.978042,0.029502,1,0.981481,0.972477,0.972477,0.990826,0.972477,0.977948,0.007323


* Folosind `DecisionTreeClassifier`

In [121]:
parameter_grid: Dict[str, List[Any]] = {'criterion':['gini','entropy'], 'max_depth': list(range(4, 10))}
df_grid_search: pd.DataFrame = calculate_performance(model=DecisionTreeClassifier(), parameter_grid=parameter_grid, X=divorce_X, y=divorce_y)
df_grid_search.head()

Accuracy: 0.9588235294117646


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000604,0.000493,0.000400,0.000490,gini,4,"{'criterion': 'gini', 'max_depth': 4}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.000601,0.000491,0.000405,0.000496,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.928571,1.0,1.0,...,0.963492,0.032547,9,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,0.000598,0.000489,0.000521,0.000454,gini,6,"{'criterion': 'gini', 'max_depth': 6}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0.000593,0.000484,0.000199,0.000399,gini,7,"{'criterion': 'gini', 'max_depth': 7}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,0.000406,0.000497,0.000592,0.000483,gini,8,"{'criterion': 'gini', 'max_depth': 8}",0.964286,1.0,1.0,...,0.978042,0.029502,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0


* Folosind `SupportVectorMachines`

In [122]:
parameter_grid: Dict[str, List[Any]] = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001], 'kernel':['rbf']}
df_grid_search: pd.DataFrame = calculate_performance(model=SVC(), parameter_grid=parameter_grid, X=divorce_X, y=divorce_y)
df_grid_search.head()

Accuracy: 0.9764705882352942


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.001383,0.000500,0.000199,0.000399,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.5,0.518519,...,0.514815,0.007407,23,0.518519,0.513761,0.513761,0.513761,0.513761,0.514713,0.001903
1,0.001191,0.000386,0.000803,0.000402,0.1,0.1,rbf,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",0.5,0.518519,...,0.514815,0.007407,23,0.518519,0.541284,0.513761,0.513761,0.532110,0.523887,0.010992
2,0.000599,0.000489,0.000599,0.000489,0.1,0.01,rbf,"{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}",1.0,0.962963,...,0.977778,0.018144,1,0.972222,0.981651,0.972477,0.981651,0.981651,0.977931,0.004558
3,0.000992,0.000014,0.000399,0.000489,0.1,0.001,rbf,"{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}",1.0,0.962963,...,0.977778,0.018144,1,0.972222,0.981651,0.972477,0.981651,0.981651,0.977931,0.004558
4,0.001396,0.000489,0.000399,0.000489,0.1,0.0001,rbf,"{'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}",0.5,0.518519,...,0.514815,0.007407,23,0.518519,0.513761,0.513761,0.513761,0.513761,0.514713,0.001903


* Folosind `SochasticGradientDescent`

In [123]:
parameter_grid: Dict[str, List[Any]] = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'penalty':['l2', 'l1', 'elasticnet']}
df_grid_search: pd.DataFrame = calculate_performance(model=SGDClassifier(), parameter_grid=parameter_grid, X=divorce_X, y=divorce_y)
df_grid_search.head()

Accuracy: 0.9705882352941178


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000605,0.000494,0.000400,0.000489,0.0001,l2,"{'alpha': 0.0001, 'penalty': 'l2'}",0.964286,0.888889,1.000000,...,0.948413,0.050320,13,1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000
1,0.000798,0.000399,0.000199,0.000397,0.0001,l1,"{'alpha': 0.0001, 'penalty': 'l1'}",0.928571,0.925926,1.000000,...,0.933862,0.036196,15,1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000
2,0.000799,0.000400,0.000199,0.000398,0.0001,elasticnet,"{'alpha': 0.0001, 'penalty': 'elasticnet'}",1.000000,0.925926,1.000000,...,0.962963,0.033127,11,1.0,0.990826,1.000000,1.0,1.0,0.998165,0.00367
3,0.000600,0.000490,0.000000,0.000000,0.001,l2,"{'alpha': 0.001, 'penalty': 'l2'}",1.000000,0.925926,1.000000,...,0.962963,0.046849,11,1.0,1.000000,0.990826,1.0,1.0,0.998165,0.00367
4,0.000996,0.000020,0.000200,0.000399,0.001,l1,"{'alpha': 0.001, 'penalty': 'l1'}",0.964286,0.925926,0.962963,...,0.941005,0.029832,14,1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000


* Folosind `RandomForrestClassifier`

In [124]:
parameter_grid: Dict[str, List[Any]] = {'n_estimators':[10, 20, 30, 100], 'criterion':['gini','entropy']}
df_grid_search: pd.DataFrame = calculate_performance(model=RandomForestClassifier(), parameter_grid=parameter_grid, X=divorce_X, y=divorce_y)
df_grid_search.head()

Accuracy: 0.9764705882352942


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.010179,0.000396,0.000996,0.000002,gini,10,"{'criterion': 'gini', 'n_estimators': 10}",0.928571,1.0,1.0,...,0.978307,0.028708,1,1.0,0.990826,0.990826,1.000000,1.000000,0.996330,0.004494
1,0.020354,0.001186,0.001604,0.000482,gini,20,"{'criterion': 'gini', 'n_estimators': 20}",0.928571,1.0,1.0,...,0.978307,0.028708,1,1.0,1.000000,1.000000,1.000000,0.990826,0.998165,0.003670
2,0.030926,0.001308,0.002935,0.000106,gini,30,"{'criterion': 'gini', 'n_estimators': 30}",0.928571,1.0,1.0,...,0.978307,0.028708,1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.094601,0.004657,0.007187,0.000396,gini,100,"{'criterion': 'gini', 'n_estimators': 100}",0.928571,1.0,1.0,...,0.978307,0.028708,1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.009973,0.000020,0.001003,0.000015,entropy,10,"{'criterion': 'entropy', 'n_estimators': 10}",0.928571,1.0,1.0,...,0.978307,0.028708,1,1.0,1.000000,0.990826,0.990826,0.990826,0.994495,0.004494


### Setul de date `HCV`

In [125]:
data_HCV: pd.DataFrame = pd.read_csv('data/hcv_data.csv')
data_HCV = data_HCV.drop('Unnamed: 0', axis=1)

labelencoder: LabelEncoder = LabelEncoder()

data_HCV.Category: pd.Series = data_HCV.Category.astype('category')
data_HCV.Category = data_HCV.Category.cat.codes
data_HCV.Sex: pd.Series = data_HCV.Sex.astype('category')
data_HCV.Sex = data_HCV.Sex.cat.codes

data_HCV = data_HCV.fillna(data_HCV.mean())

HCV_X: np.ndarray = data_HCV.iloc[:, 1:].values
HCV_y: np.ndarray = data_HCV.iloc[:, 0].values

np.any(np.isnan(data_HCV))

False

* Folosind `KNeighborsClassifier`

In [126]:
parameter_grid: Dict[str, List[Any]] = {'n_neighbors': list(range(1, 10)), 'p':[1, 2, 3, 10]}
df_grid_search: pd.DataFrame = calculate_performance(model=KNeighborsClassifier(), parameter_grid=parameter_grid, X=HCV_X, y=HCV_y)
df_grid_search.head()

Accuracy: 0.9040650406504065


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000998,2.336015e-07,0.003390,0.000488,1,1,"{'n_neighbors': 1, 'p': 1}",0.888889,0.909091,0.908163,...,0.898371,0.009095,6,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000
1,0.000797,3.986374e-04,0.003192,0.000399,1,2,"{'n_neighbors': 1, 'p': 2}",0.888889,0.898990,0.918367,...,0.902474,0.010020,2,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000
2,0.000798,3.992323e-04,0.010970,0.002092,1,3,"{'n_neighbors': 1, 'p': 3}",0.888889,0.898990,0.928571,...,0.904515,0.013489,1,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000
3,0.001197,3.989006e-04,0.008583,0.000493,1,10,"{'n_neighbors': 1, 'p': 10}",0.878788,0.898990,0.928571,...,0.898413,0.016786,4,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000
4,0.000798,3.991129e-04,0.003789,0.000399,2,1,"{'n_neighbors': 2, 'p': 1}",0.878788,0.909091,0.887755,...,0.894310,0.016505,10,0.951654,0.941476,0.93401,0.946701,0.944162,0.943601,0.005853


* Folosind `DecisionTreeClassifier`

In [127]:
parameter_grid: Dict[str, List[Any]] = {'criterion':['gini','entropy'], 'max_depth': list(range(4, 10))}
df_grid_search: pd.DataFrame = calculate_performance(model=DecisionTreeClassifier(), parameter_grid=parameter_grid, X=HCV_X, y=HCV_y)
df_grid_search.head()

Accuracy: 0.8910569105691056


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.001396,0.000489,0.000399,0.000489,gini,4,"{'criterion': 'gini', 'max_depth': 4}",0.909091,0.909091,0.918367,...,0.912616,0.010270,5,0.966921,0.959288,0.964467,0.956853,0.972081,0.963922,0.005428
1,0.001595,0.000488,0.000000,0.000000,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.909091,0.909091,0.918367,...,0.908534,0.006470,9,0.977099,0.977099,0.969543,0.974619,0.984772,0.976626,0.004919
2,0.001796,0.000399,0.000199,0.000399,gini,6,"{'criterion': 'gini', 'max_depth': 6}",0.929293,0.909091,0.918367,...,0.910534,0.012096,8,0.989822,0.989822,0.987310,0.984772,0.992386,0.988822,0.002584
3,0.001790,0.000380,0.000000,0.000000,gini,7,"{'criterion': 'gini', 'max_depth': 7}",0.919192,0.909091,0.918367,...,0.916677,0.003806,1,0.994911,0.997455,0.989848,0.987310,0.994924,0.992890,0.003727
4,0.001995,0.000001,0.000000,0.000000,gini,8,"{'criterion': 'gini', 'max_depth': 8}",0.898990,0.919192,0.928571,...,0.912616,0.010229,5,0.997455,1.000000,0.989848,0.994924,1.000000,0.996445,0.003798


* Folosind `SupportVectorMachines`

In [128]:
parameter_grid: Dict[str, List[Any]] = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001], 'kernel':['rbf']}
df_grid_search: pd.DataFrame = calculate_performance(model=SVC(), parameter_grid=parameter_grid, X=HCV_X, y=HCV_y)
df_grid_search.head()

Accuracy: 0.9089430894308943


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.038767,0.001801,0.005891,0.000498,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.858586,0.868687,...,0.865863,0.003675,9,0.867684,0.86514,0.865482,0.865482,0.865482,0.865854,0.000925
1,0.027717,0.002221,0.004982,0.000631,0.1,0.1,rbf,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",0.858586,0.868687,...,0.865863,0.003675,9,0.867684,0.86514,0.865482,0.865482,0.865482,0.865854,0.000925
2,0.021937,0.000624,0.004191,0.000400,0.1,0.01,rbf,"{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}",0.858586,0.868687,...,0.865863,0.003675,9,0.867684,0.86514,0.865482,0.865482,0.865482,0.865854,0.000925
3,0.005993,0.000027,0.001811,0.000390,0.1,0.001,rbf,"{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}",0.858586,0.868687,...,0.865863,0.003675,9,0.867684,0.86514,0.865482,0.865482,0.865482,0.865854,0.000925
4,0.002987,0.000012,0.001193,0.000401,0.1,0.0001,rbf,"{'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}",0.858586,0.868687,...,0.865863,0.003675,9,0.867684,0.86514,0.865482,0.865482,0.865482,0.865854,0.000925


* Folosind `SochasticGradientDescent`

In [129]:
parameter_grid: Dict[str, List[Any]] = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'penalty':['l2', 'l1', 'elasticnet']}
df_grid_search: pd.DataFrame = calculate_performance(model=SGDClassifier(), parameter_grid=parameter_grid, X=HCV_X, y=HCV_y)
df_grid_search.head()

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

Accuracy: 0.9170731707317075


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.003392,0.000488,0.000000,0.000000,0.0001,l2,"{'alpha': 0.0001, 'penalty': 'l2'}",0.898990,0.949495,0.887755,...,0.910513,0.021861,2,0.926209,0.933842,0.883249,0.883249,0.911168,0.907543,0.021136
1,0.005579,0.000484,0.000399,0.000488,0.0001,l1,"{'alpha': 0.0001, 'penalty': 'l1'}",0.888889,0.919192,0.918367,...,0.906514,0.011728,6,0.941476,0.913486,0.944162,0.934010,0.911168,0.928860,0.013923
2,0.004986,0.000631,0.000199,0.000399,0.0001,elasticnet,"{'alpha': 0.0001, 'penalty': 'elasticnet'}",0.868687,0.858586,0.877551,...,0.876067,0.011274,15,0.918575,0.903308,0.870558,0.926396,0.893401,0.902448,0.019663
3,0.003983,0.000012,0.000000,0.000000,0.001,l2,"{'alpha': 0.001, 'penalty': 'l2'}",0.888889,0.919192,0.908163,...,0.902432,0.016554,9,0.921120,0.921120,0.916244,0.906091,0.923858,0.917686,0.006296
4,0.006581,0.000798,0.000000,0.000000,0.001,l1,"{'alpha': 0.001, 'penalty': 'l1'}",0.898990,0.919192,0.908163,...,0.908534,0.014409,3,0.949109,0.926209,0.934010,0.936548,0.921320,0.933439,0.009536


* Folosind `RandomForrestClassifier`

In [130]:
parameter_grid: Dict[str, List[Any]] = {'n_estimators':[10, 20, 30, 100], 'criterion':['gini','entropy']}
df_grid_search: pd.DataFrame = calculate_performance(model=RandomForestClassifier(), parameter_grid=parameter_grid, X=HCV_X, y=HCV_y)
df_grid_search.head()

Accuracy: 0.9138211382113821


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.012772,0.000403,0.001189,3.865741e-04,gini,10,"{'criterion': 'gini', 'n_estimators': 10}",0.909091,0.888889,0.928571,...,0.922861,0.021502,3,0.994911,0.994911,0.989848,0.997462,0.997462,0.994919,0.002780
1,0.024951,0.000604,0.001981,1.586944e-05,gini,20,"{'criterion': 'gini', 'n_estimators': 20}",0.898990,0.919192,0.959184,...,0.926902,0.020467,1,1.000000,0.997455,1.000000,1.000000,0.997462,0.998983,0.001245
2,0.038512,0.001523,0.002793,3.994987e-04,gini,30,"{'criterion': 'gini', 'n_estimators': 30}",0.898990,0.898990,0.959184,...,0.924902,0.025062,2,1.000000,1.000000,1.000000,0.997462,1.000000,0.999492,0.001015
3,0.120827,0.003682,0.007978,9.536743e-08,gini,100,"{'criterion': 'gini', 'n_estimators': 100}",0.898990,0.909091,0.959184,...,0.922841,0.020647,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.014162,0.000747,0.001197,3.989221e-04,entropy,10,"{'criterion': 'entropy', 'n_estimators': 10}",0.909091,0.898990,0.928571,...,0.922841,0.016116,4,0.992366,1.000000,0.994924,1.000000,0.992386,0.995935,0.003447


### Setul de date `Conventional Social Media Movies`

In [131]:
from missingpy import MissForest
CSM_data: pd.DataFrame = pd.read_excel('data/2014 and 2015 CSM dataset.xlsx', index_col=None, header=None)
CSM_data = CSM_data.drop(0, axis=0)
CSM_data = CSM_data.drop(0, axis=1)

imputer: MissForest = MissForest()
CSM_data: np.ndarray = imputer.fit_transform(CSM_data)
assert not np.isnan(CSM_data.sum()) # verificam ca nu exista niciun nan

CSM_y: np.ndarray = CSM_data[:, 2]
CSM_X: np.ndarray = CSM_data[:, [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

Iteration: 0
Iteration: 1
Iteration: 2


* Folosind `KNeighborsClassifier`

In [132]:
parameter_grid: Dict[str, List[Any]] = {'n_neighbors': list(range(1, 10)), 'p':[1, 2, 3, 10]}
df_grid_search: pd.DataFrame = calculate_performance(model=KNeighborsClassifier(), parameter_grid=parameter_grid, X=CSM_X, y=CSM_y)
df_grid_search.head()

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_

Accuracy: 0.29861239592969474


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000798,0.000399,0.000998,0.000001,1,1,"{'n_neighbors': 1, 'p': 1}",0.216216,0.162162,0.162162,...,0.195796,0.027549,36,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.000810,0.000373,0.001184,0.000374,1,2,"{'n_neighbors': 1, 'p': 2}",0.270270,0.135135,0.189189,...,0.201201,0.044420,35,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,0.000600,0.000490,0.001988,0.000015,1,3,"{'n_neighbors': 1, 'p': 3}",0.297297,0.162162,0.216216,...,0.228378,0.044492,33,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.000598,0.000488,0.002006,0.000016,1,10,"{'n_neighbors': 1, 'p': 10}",0.270270,0.162162,0.216216,...,0.222973,0.036761,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.000599,0.000489,0.001281,0.000403,2,1,"{'n_neighbors': 2, 'p': 1}",0.405405,0.189189,0.216216,...,0.315465,0.093870,27,0.578231,0.598639,0.598639,0.612245,0.587838,0.595119,0.011457


* Folosind `DecisionTreeClassifier`

In [133]:
parameter_grid: Dict[str, List[Any]] = {'criterion':['gini','entropy'], 'max_depth': list(range(4, 10))}
df_grid_search: pd.DataFrame = calculate_performance(model=DecisionTreeClassifier(), parameter_grid=parameter_grid, X=CSM_X, y=CSM_y)
df_grid_search.head()

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_

Accuracy: 0.3416281221091582


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000992,0.000013,0.000000,0.000000,gini,4,"{'criterion': 'gini', 'max_depth': 4}",0.378378,0.459459,0.270270,...,0.336637,0.072449,3,0.496599,0.557823,0.537415,0.530612,0.567568,0.538003,0.024642
1,0.000793,0.000397,0.000403,0.000494,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.324324,0.405405,0.243243,...,0.315465,0.064317,5,0.578231,0.632653,0.605442,0.578231,0.648649,0.608641,0.028415
2,0.000997,0.000020,0.000200,0.000399,gini,6,"{'criterion': 'gini', 'max_depth': 6}",0.243243,0.405405,0.378378,...,0.347898,0.055453,1,0.639456,0.700680,0.693878,0.707483,0.756757,0.699651,0.037364
3,0.000798,0.000399,0.000398,0.000487,gini,7,"{'criterion': 'gini', 'max_depth': 7}",0.297297,0.378378,0.351351,...,0.342342,0.026725,2,0.700680,0.802721,0.748299,0.734694,0.810811,0.759441,0.041717
4,0.000985,0.000015,0.000199,0.000399,gini,8,"{'criterion': 'gini', 'max_depth': 8}",0.297297,0.351351,0.378378,...,0.325826,0.036161,4,0.761905,0.877551,0.775510,0.809524,0.864865,0.817871,0.046404


* Folosind `SupportVectorMachines`

In [134]:
parameter_grid: Dict[str, List[Any]] = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001], 'kernel':['rbf']}
df_grid_search: pd.DataFrame = calculate_performance(model=SVC(), parameter_grid=parameter_grid, X=CSM_X, y=CSM_y)
df_grid_search.head()

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_

Accuracy: 0.2814061054579093


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.005596,0.000488,0.001193,3.860128e-04,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.27027,0.297297,...,0.282583,0.012323,1,0.285714,0.278912,0.278912,0.285714,0.283784,0.282607,0.003099
1,0.005810,0.000417,0.001189,4.039721e-04,0.1,0.1,rbf,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",0.27027,0.297297,...,0.282583,0.012323,1,0.285714,0.278912,0.278912,0.285714,0.283784,0.282607,0.003099
2,0.005589,0.000481,0.001187,4.052356e-04,0.1,0.01,rbf,"{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}",0.27027,0.297297,...,0.282583,0.012323,1,0.285714,0.278912,0.278912,0.285714,0.283784,0.282607,0.003099
3,0.005386,0.000489,0.001203,3.961144e-04,0.1,0.001,rbf,"{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}",0.27027,0.297297,...,0.282583,0.012323,1,0.285714,0.278912,0.278912,0.285714,0.283784,0.282607,0.003099
4,0.005778,0.000396,0.000997,4.422006e-07,0.1,0.0001,rbf,"{'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}",0.27027,0.297297,...,0.282583,0.012323,1,0.285714,0.278912,0.278912,0.285714,0.283784,0.282607,0.003099


* Folosind `SochasticGradientDescent`

In [135]:
parameter_grid: Dict[str, List[Any]] = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'penalty':['l2', 'l1', 'elasticnet']}
df_grid_search: pd.DataFrame = calculate_performance(model=SGDClassifier(), parameter_grid=parameter_grid, X=CSM_X, y=CSM_y)
df_grid_search.head()

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum numbe

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

Accuracy: 0.3116558741905643


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\s

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.005178,0.000403,0.000000,0.000000,0.0001,l2,"{'alpha': 0.0001, 'penalty': 'l2'}",0.297297,0.162162,0.351351,...,0.244595,0.074557,12,0.319728,0.142857,0.346939,0.244898,0.229730,0.256830,0.072031
1,0.005983,0.000002,0.000399,0.000489,0.0001,l1,"{'alpha': 0.0001, 'penalty': 'l1'}",0.243243,0.243243,0.351351,...,0.315766,0.066392,1,0.204082,0.265306,0.408163,0.306122,0.331081,0.302951,0.067917
2,0.007586,0.003265,0.000000,0.000000,0.0001,elasticnet,"{'alpha': 0.0001, 'penalty': 'elasticnet'}",0.324324,0.243243,0.189189,...,0.216817,0.069561,15,0.346939,0.231293,0.183673,0.231293,0.121622,0.222964,0.073922
3,0.004788,0.000398,0.000000,0.000000,0.001,l2,"{'alpha': 0.001, 'penalty': 'l2'}",0.378378,0.216216,0.162162,...,0.245045,0.095240,11,0.387755,0.231293,0.231293,0.176871,0.263514,0.258145,0.070534
4,0.009781,0.004257,0.000206,0.000412,0.001,l1,"{'alpha': 0.001, 'penalty': 'l1'}",0.243243,0.216216,0.270270,...,0.211712,0.042256,16,0.224490,0.183673,0.394558,0.272109,0.195946,0.254155,0.076513


* Folosind `RandomForrestClassifier`

In [136]:
parameter_grid: Dict[str, List[Any]] = {'n_estimators':[10, 20, 30, 100], 'criterion':['gini','entropy']}
df_grid_search: pd.DataFrame = calculate_performance(model=RandomForestClassifier(), parameter_grid=parameter_grid, X=CSM_X, y=CSM_y)
df_grid_search.head()

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_

Accuracy: 0.3414431082331175


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.011808,0.000735,0.001189,0.000405,gini,10,"{'criterion': 'gini', 'n_estimators': 10}",0.324324,0.324324,0.351351,...,0.336937,0.012220,5,0.979592,0.979592,0.986395,0.986395,0.986486,0.983692,0.003348
1,0.023018,0.000946,0.001998,0.000621,gini,20,"{'criterion': 'gini', 'n_estimators': 20}",0.405405,0.486486,0.162162,...,0.331532,0.111275,6,1.000000,1.000000,1.000000,0.986395,1.000000,0.997279,0.005442
2,0.033613,0.001325,0.002593,0.000489,gini,30,"{'criterion': 'gini', 'n_estimators': 30}",0.459459,0.351351,0.297297,...,0.325976,0.079685,7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.111495,0.004185,0.008177,0.000399,gini,100,"{'criterion': 'gini', 'n_estimators': 100}",0.459459,0.405405,0.270270,...,0.369670,0.065795,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.012754,0.000396,0.001208,0.000394,entropy,10,"{'criterion': 'entropy', 'n_estimators': 10}",0.378378,0.378378,0.432432,...,0.347147,0.071206,4,0.986395,0.965986,0.972789,0.972789,1.000000,0.979592,0.012169


## Subpunctul 4 - Documentatie

4. Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Daca acelasi algoritm e folosit pentru mai multe seturi de date, puteti face o sectiune separata cu documentarea algoritmilor + trimitere la algoritm. 

### KNeighboursClassifier

Link catre documentatia oficiala:
[https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

Clasificarea pe baza de vecini este un tip de ‘non-generalizing learning’. Aceasta nu incearca sa construiasca un model intern, ci pur si simplu memoreaza instantele din datele de antrenare. Classificarea se face printr-o simpla numarare a vecinilor fiecarui punct. Unui punct i se desemneaza clasa care are cei mai multi reprezentanti printre vecinii apropiati ai acestui punct.

Clasificarea prin KNeighborsClassifier este una dintre cele mai utilizate tehnici. Alegerea optima a valorii lui k este dependenta in mare parte de date. In general, un k mai mare suprima efectele zgomotului, dar face limitele clasificarii mai putin distincte.

In mod normal, clasificarea foloseste weight-uri uniforme, astfel valoarea atribuita unui punct este calculate printr-o simpla numaratoare a vecinilor apropiati. In unele cazuri, este mai bine sa masuram vecinii astfel incat vecinii mai apropiati sa contribuie mai mult. Pentru a reusi asta, putem modifica parametrul ‘weights’. Valoarea default, *weights=’uniform’* atribuie valori uniforme fiecarui punct. Daca folosim *weights=’distance’* , atunci se atribuie valori proportionale cu inversul distantei de la punctul respectiv. 


<img src="./images/sphx_glr_plot_classification_001.png" alt="drawing" width="600px"/>
<img src="./images/sphx_glr_plot_classification_002.png" alt="drawing" width="600px"/>

### DecisionTreeClassifier

Link catre documentatia oficiala: [https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Arborii de decizie (DT) sunt o metodă de învățare supravegheată non-parametrică utilizată pentru clasificare și regresie. Arborii de decizie învață din date pentru a aproxima o curbă sinusoidală cu un set de reguli de decizie if-then-else. Cu cât arborele este mai adânc, cu atât regulile de decizie sunt mai complexe și modelul este mai potrivit.
Arborele decizional construiește modele de clasificare sau regresie sub forma unei structuri arborescente. Descompune un set de date în subseturi din ce în ce mai mici, în timp ce în același timp este dezvoltat incremental un arbore de decizie asociat. Rezultatul final este un arbore cu noduri de decizie și noduri de frunze. Un nod de decizie are două sau mai multe ramuri. Nodul frunzei reprezintă o clasificare sau o decizie. Cel mai înalt nod de decizie dintr-un arbore care corespunde celui mai bun predictor numit nod rădăcină. Arborii de decizie pot gestiona atât date categorice cât și date numerice.

<img src="./images/1_UkQ2GHsopx3P1AiGWN7f0Q.png" alt="drawing" width="600px"/>

### SupportVectorMachine

Link catre documentatia oficiala: [https://scikit-learn.org/stable/modules/svm.html](https://scikit-learn.org/stable/modules/svm.html)

„Support Vector Machine” (SVM) este un algoritm de învățare automată supravegheata care poate fi utilizat atât pentru probleme de clasificare, fie pentru regresie. Cu toate acestea, este utilizat mai ales în probleme de clasificare. În algoritmul SVM, reprezentăm fiecare element de date ca un punct din spațiul n-dimensional (unde n este numărul de caracteristici pe care le aveți), valoarea fiecărei caracteristici fiind valoarea unei anumite coordonate. Apoi, efectuăm clasificarea găsind hiper-planul care diferențiază foarte bine cele două clase.

<img src="./images/SVM_1.png" alt="drawing" width="400px"/>

Support Vectors sunt coordonatele observației individuale. Clasificatorul SVM este o frontieră care separă cel mai bine cele două clase (hiper-plan / linie).

### SochasticGradientDescent

Link catre documentatia oficiala: [https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)

Gradient Descent este un algoritm de optimizare utilizat pentru a găsi valorile parametrilor (coeficienților) unei funcții (f) care minimizează o funcție de cost (cost).

Gradient descent este cel mai bine utilizată atunci când parametrii nu pot fi calculați analitic (de exemplu, folosind algebra liniară) și trebuie căutați de un algoritm de optimizare.

**Intuitia pentru coborârea gradientului**

Gândiți-vă la un castron mare, cum ar fi ceea ce ați mânca din cereale sau ați păstra fructe. Acest castron este un complot al funcției de cost (f).

<img src="./images/Large-Bowl.jpg" alt="drawing" width="400px"/>

O poziție aleatorie pe suprafața bolului este costul valorilor actuale ale coeficienților (cost).

Fundul bolului este costul celui mai bun set de coeficienți, minimul funcției.

Scopul este de a încerca în continuare valori diferite pentru coeficienți, de a evalua costul acestora și de a selecta noi coeficienți care au un cost puțin mai bun (mai mic).

Repetarea acestui proces de suficiente ori va duce la partea de jos a vasului și veți cunoaște valorile coeficienților care duc la costul minim.

**SochasticGradientDescent**

Gradient Descent poate fi lentă pentru a rula pe seturi de date foarte mari.

Deoarece o iterație a algoritmului de Gradient Descent necesită o predicție pentru fiecare instanță din setul de date de antrenament, poate dura mult timp când aveți multe milioane de instanțe.

În situațiile în care aveți cantități mari de date, puteți utiliza o variație a coborârii în gradient numită Sochastic Gradient Descent.

În această variantă, se execută procedura de Gradient Descent descrisă mai sus, dar actualizarea coeficienților se efectuează pentru fiecare instanță de antrenament, mai degrabă decât la sfârșitul lotului de instanțe.

Primul pas al procedurii necesită ca ordinea setului de date de instruire să fie randomizată. Aceasta este pentru a amesteca ordinea în care se fac actualizări ale coeficienților. Deoarece coeficienții sunt actualizați după fiecare instanță de antrenament, actualizările vor sari zgomotos peste tot, la fel și funcția de cost corespunzătoare. Amestecând ordinea actualizărilor coeficienților, utilizează această plimbare aleatorie și evită să fie distrasă sau blocată.

Procedura de actualizare a coeficienților este aceeași cu cea de mai sus, cu excepția faptului că costul nu este însumat pentru toate tiparele de antrenament, ci calculat pentru un singur model de antrenament.

Învățarea poate fi mult mai rapidă cu Sochastic Gradient Descent pentru seturi de date de antrenament foarte mari și de multe ori aveți nevoie doar de un număr mic de treceri prin setul de date pentru a ajunge la un set suficient sau bun de coeficienți.

### RandomForestClassifier

Link catre documentatia oficiala: [https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Random Forest este un algoritm de învățare supervizata. „Pădurea” pe care o construiește este un ansamblu de arbori de decizie, de obicei instruiți cu metoda „împachetării”. Ideea generală a metodei de împachetare este că o combinație de modele de învățare crește rezultatul general. 

**Random Forest construiește mai mulți arbori de decizie și îi combină pentru a obține o predicție mai precisă și stabilă.**

Un mare avantaj al Random Forest este că poate fi utilizat atât pentru probleme de clasificare, cât și pentru probleme de regresie, care formează majoritatea sistemelor actuale de învățare automată. Să vedem Random Forest în clasificare, deoarece clasificarea este uneori considerată elementul de bază al învățării automate. Mai jos puteți vedea cum ar arăta o Random Forest cu doi copaci:

<img src="./images/two-tree-random-forest.png" alt="drawing" width="400px"/>

Random Forest are aproape aceiași hiperparametri ca un Decision Tree. Cu Random Forest, puteți face față și sarcinilor de regresie utilizând regresorul algoritmului. Random Forest adaugă randomizarea suplimentară modelului, în timp ce crește copacii. În loc să caute cea mai importantă caracteristică în timp ce împarte un nod, caută cea mai bună caracteristică dintr-un subset de caracteristici aleatorii. Acest lucru are ca rezultat o mare diversitate care, în general, duce la un model mai bun.

Prin urmare, în Random Forest, doar un subset aleatoriu de caracteristici este luat în considerare de algoritmul pentru împărțirea unui nod. Puteți chiar să faceți copacii mai aleatori utilizând suplimentar praguri aleatorii pentru fiecare caracteristică, mai degrabă decât căutând cele mai bune praguri posibile (așa cum o face un arbore de decizie normal).

### GaussianProcessClassifier

Link catre documentatia oficiala: [https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html)

Gaussian Process Classifier este un algoritm de învățare automată a clasificării.

Procesele Gaussiene sunt o generalizare a distribuției probabilitatii Gaussiene și pot fi utilizate ca bază pentru algoritmi sofisticati de învățare automată neparametrică pentru clasificare și regresie.

Acestea sunt un tip de model de nucleu, la fel ca SVM-urile și, spre deosebire de SVM-uri, sunt capabile să prezică probabilități de calificare a calității de membru foarte bine calibrate, deși alegerea și configurarea nucleului utilizat în centrul metodei pot fi dificile.

Un **nucleu** este o funcție care măsoară asemănarea dintre două intrări. Valori mai mari = mai asemănătoare.

Nucleul RBF definește asemănarea ca fiind distanța eudliceană între cele două intrări (similar cu cel mai apropiat vecin). Dacă cele două intrări sunt corecte între ele, obțin similaritatea maximă de 1. Dacă sunt „prea departe” unul de celălalt, nucleul RBF spune doar că nu sunt similare (returnând o valoare aproape de 0).

<img src="./images/sphx_glr_plot_gpc_iris_001.png" alt="drawing" width="600px"/>